This notebooks help to prepare dataset for training flow matching model.

We convert images to VAE latents and saves them locally

In [1]:
#folder with images dataset
images_folder='/home/vlad/Documents/data'

# where to save VAE latents
save_latents_dir = './latents'

# image size
image_size=256

# how many dataset repeats to prepare
repeats = 1

In [2]:
import os
import torchvision.transforms as T
from kemsekov_torch.utils import PadToMultiple
import tqdm
from vae import *

tr = T.Compose([
    T.ToTensor(),
    T.Resize(image_size),
    T.RandomHorizontalFlip(),
    PadToMultiple(image_size//4),
])

image_formats=['.png','.jpg','.jpeg','.webp']
for dir_path,subf,files in os.walk(images_folder):
    files = [os.path.join(dir_path,f) for f in files if [f.endswith(v) for v in image_formats]]
    class_ = os.path.split(dir_path)[-1]
    if len(files)==0: continue
    save_folder = os.path.join(save_latents_dir,class_)
    os.makedirs(save_folder,exist_ok=True)
    files_r = files*repeats
    data = tqdm.tqdm(enumerate(files_r),total=len(files_r),desc=class_)
    for i,f in data:
        latents = encode(tr(Image.open(f).convert("RGB")))
        torch.save(latents[0],os.path.join(save_folder,str(i)+".pt"))

/home/vlad/Programs/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
cat: 100%|██████████| 2553/2553 [02:00<00:00, 21.18it/s]
